<h2>Implementacja modelu BCRNN do postaci Neurona</h2>

Artykuł naukowy:
Instniejace, podobne rozwiązania:

In [ ]:
import numpy, math, os
import torch.nn as nn
from torch import Tensor

In [ ]:
__all__ = [
    'HCR'
]

In [ ]:
from torch.nn import LSTM, Module

'''
Klasa rozszerzająca funkcje torch'a.
'''

class HCRNN_Neuron(nn.Module):

    '''
    definicja stałych określająca rozmiar wejścia i wyjścia dla liczb neuronów.
    '''

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int

    def __init__(
        self,
        in_features: int,
        out_features: int,
        
        #definicja urządzenia, CPU, GPU lub TPU
        device = None,
        dtype= None,

        hierarchy_levels:int=1

    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.hierarchy_levels = hierarchy_levels
        self.hidden_size = out_features
        
        self.level1_rnn = nn.LSTM(in_features, out_features, batch_first=True)
        
        self.level2_rnn = nn.LSTM(out_features, out_features, batch_first=True)
                
        # Rekonstrukcja końcowego outputu
        self.reconstruct = nn.Linear(out_features, in_features)

    #Tensor wchodzi i zawsze Tensor wychodzi
    def forward(self, input:Tensor) -> Tensor:
        out1, _ = self.level1_rnn(x)
        out2, _ = self.level2_rnn(out1)
        
        reconstruction = self.reconstruct(out2)
        return reconstruction

In [2]:
import torch.nn as nn

In [ ]:
nn.Linear(10, 20)
HCRNN_Neuron(10, 20)